# DmfA XML to RDF

## Imports

In [ ]:
import os
import subprocess
import ipywidgets as widgets
from IPython.display import FileLink
from SPARQLWrapper import SPARQLWrapper, TURTLE

## Input file to tranform

Filepath

In [ ]:
filepath = input() # ex: ./DMFA/DmfAExamples/Linked_DmfAOriginal_20223_1_Corrected.xml

if os.path.isfile(filepath):
    print("File exists")    
else:
    print("File does not exist")

File Upload

In [ ]:
uploader = widgets.FileUpload(multiple=False)
display(uploader)

In [ ]:
uploadsDir = "uploads"

if not os.path.exists(uploadsDir):
    os.makedirs(uploadsDir)

if uploader.value:
    fileDict = next(iter(uploader.value))
    filename = fileDict["name"]
    fileContent = fileDict["content"]

    filepath = os.path.join(uploadsDir, filename)
    with open(filepath, "wb") as file:
        file.write(fileContent)

## Input output file name

In [ ]:
outfilename = input() #ex: valid_dmfa.ttl

## Input reference (optional)

In [ ]:
reference = input() #ex: 000, 111

if not reference:
    reference = "NOREF"

## Input triple store endpoint

In [ ]:
endpoint = "http://localhost:3030/socialsecurity"

## Tranformation

In [ ]:
# SPARQL endpoint setup
sparql = SPARQLWrapper(endpoint)
sparql.setReturnFormat(TURTLE)

Fetch mappings graph


In [ ]:
# Create mapping annexes
FETCH_MAPPING_QUERY = """
PREFIX rml: <http://semweb.mmlab.be/ns/rml#>
PREFIX rr: <http://www.w3.org/ns/r2rml#>

CONSTRUCT {{
  ?s ?p ?o.
  ?LogicalSource rml:source "{0}" .
  ?sm rr:template ?newtemplate .
}}
FROM <http://kg.socialsecurity.be/resource/mappings/dmfaxml/>
WHERE {{
  {{
    ?LogicalSource rml:source ?source.
  }}
  UNION
  {{
    ?tm rr:subjectMap ?sm.
    ?sm rr:template ?template.
    BIND ( REPLACE(?template, "NOREF", "{1}") as ?newtemplate)  
  }}
  UNION
  {{
    ?s ?p ?o.
    FILTER (?p != rml:source)
    MINUS {{
      ?tm rr:subjectMap ?s.
      ?s rr:template ?o.
    }}
  }}
}}
"""

sparql.setQuery(FETCH_MAPPING_QUERY.format(filepath, reference))
response = sparql.queryAndConvert().decode("utf-8")

mapppingsFilename = "mappingsTmp.ttl"
mapppings = open(mapppingsFilename, "w", encoding="utf-8")
mapppings.write("\n" + response)
mapppings.close()

Create RDF graph with RMLMapper

In [ ]:
downloadsDir = "downloads"

if not os.path.exists(downloadsDir):
    os.makedirs(downloadsDir)

outfilepath = os.path.join(downloadsDir, outfilename)

command = "java -jar ./package/rmlmapper/rmlmapper.jar -m {}  > {}".format(mapppingsFilename, outfilepath)
subprocess.run(command, shell=True)

download_link = FileLink(outfilepath, result_html_prefix="Click here to download: ")
display(download_link)

## Clean up

In [ ]:
os.remove(mapppingsFilename)